## 모듈 불러오기

In [5]:
import sys

print(sys.version)

# 파이썬 버전이 3.10.14여야 합니다

import torch

print(torch.cuda.is_available())

# CUDA 사용이 반드시 가능해야합니다.

3.10.14 | packaged by Anaconda, Inc. | (main, May  6 2024, 19:44:50) [MSC v.1916 64 bit (AMD64)]
True


In [6]:
!pip install sdv==0.18.0

INFO: pip is looking at multiple versions of rdt to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of rdt to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   -------------------------------------- - 1.6/1.6 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 6.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ----- ---------------------------------- 2.4/15.8 MB 11.2 MB/s eta 0:00:02
   ----------- ---------------------------- 4.5/15.8 MB 10.8 MB/s eta 0:00:02
   ----------------- ---------------------- 6.8/15.8 MB 11.1 MB/s eta 0:00:01
   --------------------- ------------------ 8.7/15.8 MB 11.0 MB/s eta 0:00:01
   -------------------------- ------------- 10.5/15.8 MB 10.2 MB/s eta 0:00:01
   ----------------------------

  You can safely remove it manually.
  You can safely remove it manually.


In [7]:
!pip install gretel-synthetics

INFO: pip is looking at multiple versions of gretel-synthetics to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of gretel-synthetics to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ------------------------------------- -- 1.0/1.1 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ----------- ---------------------------- 1.8/6.3 

In [14]:
%pip install pandas==1.5.3 numpy==1.26.4

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import warnings

warnings.filterwarnings("ignore")

import zipfile

from sklearn.metrics import f1_score, classification_report
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

from gretel_synthetics.actgan import ACTGAN
from tqdm import tqdm

In [5]:
file_path = r'C:\Users\james\Desktop\2024-8'

### 생성 전 전처리

In [6]:
train_all = pd.read_csv(file_path + r'\train.csv')
train = train_all.drop('ID', axis=1, inplace=True)

In [11]:
# train['35/34'] = train.apply(lambda row: row['Account_one_month_std_dev'] / row['Account_one_month_max_amount'] if row['Account_one_month_max_amount'] != 0 else 0, axis=1)
# train['37/36'] = train.apply(lambda row: row['Account_dawn_one_month_std_dev'] / row['Account_dawn_one_month_max_amount'] if row['Account_dawn_one_month_max_amount'] != 0 else 0, axis=1)
# train['36/34'] = train.apply(lambda row: row['Account_dawn_one_month_max_amount'] / row['Account_one_month_max_amount'] if row['Account_one_month_max_amount'] != 0 else 0, axis=1)

In [12]:
# train['7_29_30'] = train['Customer_credit_rating'] + '_' + train['Account_indicator_release_limit_excess'].astype(str) +'_' + train['Account_amount_daily_limit'].astype(str)
# train['7_29_30'] = train['Customer_credit_rating'] + '_' + train['Account_indicator_release_limit_excess'].astype(str) +'_' + train['Account_amount_daily_limit'].astype(str)

# train['40_41'] = train['Channel'] + '_' + train['Operating_System']

In [13]:
# def convert_to_float_year(dt):
#         year = dt.year
#         start_of_year = pd.Timestamp(year=year, month=1, day=1)
#         next_year = pd.Timestamp(year=year + 1, month=1, day=1)
#         year_elapsed = (dt - start_of_year).total_seconds()
#         year_duration = (next_year - start_of_year).total_seconds()
#         fractional_year = year_elapsed / year_duration
#         return year + fractional_year

# train['Account_creation_datetime'] = pd.to_datetime(train['Account_creation_datetime'])
# train['Transaction_Datetime'] = pd.to_datetime(train['Transaction_Datetime'])
# train['Last_atm_transaction_datetime'] = pd.to_datetime(train['Last_atm_transaction_datetime'])
# train['Last_bank_branch_transaction_datetime'] = pd.to_datetime(train['Last_bank_branch_transaction_datetime'])
# train['Transaction_resumed_date'] = pd.to_datetime(train['Transaction_resumed_date'])
# train['Customer_registration_datetime'] = pd.to_datetime(train['Customer_registration_datetime'])

# train['계좌개설일자_소수'] = train['Account_creation_datetime'].apply(convert_to_float_year)
# train['거래재개일자_소수'] = train['Transaction_resumed_date'].apply(convert_to_float_year)
# train['거래일자_소수'] = train['Transaction_Datetime'].apply(convert_to_float_year)
# train['마지막ATM거래일자_소수'] = train['Last_atm_transaction_datetime'].apply(convert_to_float_year)
# train['마지막영업점거래일자_소수'] = train['Last_bank_branch_transaction_datetime'].apply(convert_to_float_year)
# train['등록일자_소수'] = train['Customer_registration_datetime'].apply(convert_to_float_year)

# train['거래까지걸린시간_소수'] = train['거래일자_소수'] - train['거래재개일자_소수']
# train['거래까지걸린시간_ATM_소수'] = train['거래일자_소수'] - train['마지막ATM거래일자_소수']
# train['거래까지걸린시간_영업점_소수'] = train['거래일자_소수'] - train['마지막영업점거래일자_소수']
# train['거래당시연차'] = train['거래일자_소수'] - train['계좌개설일자_소수']
# train['등록부터 계좌개설까지 걸린시간_소수'] = train['계좌개설일자_소수'] - train['등록일자_소수']
# train['등록당시나이'] = train['등록일자_소수'] - train['Customer_Birthyear']

In [14]:
# drop_columns = ['Customer_personal_identifier', 'Customer_identification_number', 'Account_account_number', 'Location', 'IP_Address', 'MAC_Address', 'Recipient_Account_Number', # 학습 원하지 않는 컬럼들
#                 'Account_one_month_std_dev', 'Account_dawn_one_month_std_dev', 'Account_dawn_one_month_max_amount', # -> 비율로 학습할 컬럼들
#                 'Customer_credit_rating', 'Account_indicator_release_limit_excess', 'Account_amount_daily_limit',
#                 'Channel', 'Operating_System',
#                 'Transaction_Failure_Status', # -> 조합이 고정되는 컬럼들
#                 'Customer_registration_datetime', 'Account_creation_datetime', 'Transaction_Datetime', 'Last_atm_transaction_datetime', 'Last_bank_branch_transaction_datetime', 'Transaction_resumed_date',
#                 '계좌개설일자_소수', '거래재개일자_소수', '거래일자_소수', '마지막ATM거래일자_소수', '마지막영업점거래일자_소수', '등록일자_소수' , # -> 소수 변환 컬럼들
#                 ]

# train.drop(columns=drop_columns, inplace=True)

In [15]:
# def handle_outliers(series, n_std=3):
#     mean = series.mean()
#     std = series.std()

#     # z-score 계산
#     z_scores = (series - mean) / std

#     # n_std를 넘는 z-score를 가진 값을 n_std에 해당하는 원본 값으로 대체
#     series = series.mask(z_scores > n_std, mean + n_std * std)
#     series = series.mask(z_scores < -n_std, mean - n_std * std)

#     return series

# train['Time_difference_seconds'] = pd.to_timedelta(train['Time_difference']).dt.total_seconds()

# ## 이상치 처리 - 시간

# # 거래까지걸린시간_소수의 평균값 계산
# mean_value = train['거래까지걸린시간_소수'].mean()
# train.loc[train['Time_difference_seconds'] > 31536000, '거래까지걸린시간_소수'] = mean_value

# mean_value = train['Time_difference_seconds'].mean()
# train['Time_difference_seconds'] = train['Time_difference_seconds'].apply(
#     lambda x: mean_value if x > 31536000 else x
# )

# min_positive_value = train['Time_difference_seconds'][train['Time_difference_seconds'] >= 0].min()
# train['Time_difference_seconds'] = train['Time_difference_seconds'].apply(
#     lambda x: min_positive_value if x < 0 else x
# )

# ## 이상치 처리 - 금액
# cost_cols = [
#     'Account_initial_balance', 'Account_balance', 'Account_remaining_amount_daily_limit_exceeded', 'Account_one_month_max_amount',
#     'Transaction_Amount'
# ]

# for col in cost_cols:
#     train[col] = handle_outliers(train[col])

In [16]:
# 모델 저장하는 코드

# # 모든 Fraud_Type 목록 생성 (m 포함)
# fraud_types = train['Fraud_Type'].unique()

# # 모든 합성 데이터를 저장할 DataFrame 초기화
# all_synthetic_data = pd.DataFrame()

# # 각 Fraud_Type에 대해 학습된 모델을 저장할 딕셔너리
# models_dict = {}

# # 각 Fraud_Type에 대해 합성 데이터 생성 및 저장
# for fraud_type in tqdm(fraud_types):
#     f = fraud_type
#     if fraud_type == 'm':
#         N_SAMPLE = 10000
#         N = 1000
#         EPOCH = 100
#     else:
#         N_SAMPLE = 100
#         EPOCH = 100
#         N = 1000

#     # 해당 Fraud_Type에 대한 서브셋 생성
#     subset = train[train["Fraud_Type"] == fraud_type]

#     # 모든 Fraud_Type에 대해 100개씩 샘플링
#     subset = subset.sample(n=N_SAMPLE, random_state=42)

#     # Time_difference 열 제외 (초 단위로 변환된 컬럼만 사용)
#     subset = subset.drop('Time_difference', axis=1)

#     synthesizer = ACTGAN(
#                             epochs=EPOCH,
#                             cuda=True
#                         )

#     synthesizer.fit(subset)

#     # 학습된 모델을 딕셔너리에 저장
#     models_dict[fraud_type] = synthesizer

#     synthetic_subset = synthesizer.sample(num_rows=N)

#     for col in cost_cols:
#         synthetic_subset[col] = handle_outliers(synthetic_subset[col])

#     # Time_difference_seconds를 다시 timedelta로 변환
#     synthetic_subset['Time_difference'] = pd.to_timedelta(synthetic_subset['Time_difference_seconds'], unit='s')

#     # Time_difference_seconds 컬럼 제거
#     synthetic_subset = synthetic_subset.drop('Time_difference_seconds', axis=1)

#     # 생성된 데이터를 all_synthetic_data에 추가
#     all_synthetic_data = pd.concat([all_synthetic_data, synthetic_subset], ignore_index=True)

#     # 최종 결과 확인
#     print("\nFinal All Synthetic Data Shape:", all_synthetic_data.shape)

# # 모든 모델을 하나의 .pth 파일에 저장
# torch.save(models_dict, './ACTGAN_clf_submission.pth')
# print("All models saved in ACTGAN_clf_submission.pth")

### 생성하기

In [7]:
def handle_outliers(series, n_std=3):
    mean = series.mean()
    std = series.std()

    # z-score 계산
    z_scores = (series - mean) / std

    # n_std를 넘는 z-score를 가진 값을 n_std에 해당하는 원본 값으로 대체
    series = series.mask(z_scores > n_std, mean + n_std * std)
    series = series.mask(z_scores < -n_std, mean - n_std * std)

    return series

cost_cols = [
    'Account_initial_balance', 'Account_balance', 'Account_remaining_amount_daily_limit_exceeded', 'Account_one_month_max_amount',
    'Transaction_Amount'
]

# 모델 파일 불러오기
if torch.cuda.is_available():
    models_dict = torch.load(file_path + './ACTGAN_clf_submission.pth')

# 모든 Fraud_Type 목록 생성 (m 포함)
fraud_types = train_all['Fraud_Type'].unique()

# 모든 합성 데이터를 저장할 DataFrame 초기화
all_synthetic_data = pd.DataFrame()

# 각 Fraud_Type에 대해 합성 데이터 생성 및 저장
for fraud_type in tqdm(fraud_types):
    f = fraud_type
    if fraud_type == 'm':
        N_SAMPLE = 10000
        N = 1000
    else:
        N_SAMPLE = 100
        N = 1000

    # 이미 학습된 모델을 불러오기
    synthesizer = models_dict[fraud_type]

    # 합성 데이터 생성
    synthetic_subset = synthesizer.sample(num_rows=N)

    # 생성된 데이터에 후처리 적용
    for col in cost_cols:
        synthetic_subset[col] = handle_outliers(synthetic_subset[col])

    # Time_difference_seconds를 다시 timedelta로 변환
    synthetic_subset['Time_difference'] = pd.to_timedelta(synthetic_subset['Time_difference_seconds'], unit='s')

    # Time_difference_seconds 컬럼 제거
    synthetic_subset = synthetic_subset.drop('Time_difference_seconds', axis=1)

    # 생성된 데이터를 all_synthetic_data에 추가
    all_synthetic_data = pd.concat([all_synthetic_data, synthetic_subset], ignore_index=True)

    # 최종 결과 확인
    print("\nFinal All Synthetic Data Shape:", all_synthetic_data.shape)

  8%|▊         | 1/13 [00:00<00:10,  1.18it/s]


Final All Synthetic Data Shape: (1000, 52)


 15%|█▌        | 2/13 [00:01<00:07,  1.52it/s]


Final All Synthetic Data Shape: (2000, 52)


 23%|██▎       | 3/13 [00:01<00:05,  1.81it/s]

...


Final All Synthetic Data Shape: (11000, 52)


 92%|█████████▏| 12/13 [00:05<00:00,  2.51it/s]


Final All Synthetic Data Shape: (12000, 52)


100%|██████████| 13/13 [00:05<00:00,  2.27it/s]


Final All Synthetic Data Shape: (13000, 52)


### 생성 후 전처리

In [8]:
all_synthetic_data[['Channel', 'Operating_System']] = all_synthetic_data['40_41'].str.split('_', expand=True)

all_synthetic_data[['Customer_credit_rating', 'Account_indicator_release_limit_excess', 'Account_amount_daily_limit']] = all_synthetic_data['7_29_30'].str.split('_', expand=True)
all_synthetic_data['Account_indicator_release_limit_excess'] = all_synthetic_data['Account_indicator_release_limit_excess'].astype(int)
all_synthetic_data['Account_amount_daily_limit'] = all_synthetic_data['Account_amount_daily_limit'].astype(int)

all_synthetic_data['Transaction_Failure_Status'] = all_synthetic_data['Error_Code'].apply(lambda x: 0 if x == 'a' else 1)

all_synthetic_data.drop(columns=['7_29_30', '40_41'], inplace=True)

In [9]:
all_synthetic_data['Account_one_month_std_dev'] = (all_synthetic_data['35/34']*all_synthetic_data['Account_one_month_max_amount']).astype(int)
all_synthetic_data['Account_dawn_one_month_max_amount'] = (all_synthetic_data['36/34']*all_synthetic_data['Account_one_month_max_amount']).astype(int)
all_synthetic_data['Account_dawn_one_month_std_dev'] = (all_synthetic_data['37/36']*all_synthetic_data['Account_dawn_one_month_max_amount']).astype(int)

all_synthetic_data.drop(columns=['35/34', '36/34', '37/36'], inplace=True)

In [10]:
import math
def convert_from_float_year(float_year):
    year = int(math.floor(float_year))
    fractional_part = float_year - year
    start_of_year = pd.Timestamp(year=year, month=1, day=1)
    next_year = pd.Timestamp(year=year + 1, month=1, day=1)
    year_duration = (next_year - start_of_year).total_seconds()
    elapsed_seconds = round(fractional_part * year_duration)  # 초를 반올림
    result_date = start_of_year + pd.to_timedelta(elapsed_seconds, unit='s')
    return result_date

all_synthetic_data['등록일자_소수'] = all_synthetic_data['Customer_Birthyear'] + all_synthetic_data['등록당시나이']
all_synthetic_data['계좌개설일자_소수'] = all_synthetic_data['등록일자_소수'] + all_synthetic_data['등록부터 계좌개설까지 걸린시간_소수']
all_synthetic_data['거래일자_소수'] = all_synthetic_data['계좌개설일자_소수'] + all_synthetic_data['거래당시연차']
all_synthetic_data['거래재개일자_소수'] = all_synthetic_data['거래일자_소수'] - all_synthetic_data['거래까지걸린시간_소수']
all_synthetic_data['마지막ATM거래일자_소수'] = all_synthetic_data['거래일자_소수'] - all_synthetic_data['거래까지걸린시간_ATM_소수']
all_synthetic_data['마지막영업점거래일자_소수'] = all_synthetic_data['거래일자_소수'] - all_synthetic_data['거래까지걸린시간_영업점_소수']

all_synthetic_data['Customer_registration_datetime'] = all_synthetic_data['등록일자_소수'].apply(convert_from_float_year)
all_synthetic_data['Account_creation_datetime'] = all_synthetic_data['계좌개설일자_소수'].apply(convert_from_float_year)
all_synthetic_data['Transaction_Datetime'] = all_synthetic_data['거래일자_소수'].apply(convert_from_float_year)
all_synthetic_data['Transaction_resumed_date'] = all_synthetic_data['거래재개일자_소수'].apply(convert_from_float_year)
all_synthetic_data['Last_atm_transaction_datetime'] = all_synthetic_data['마지막ATM거래일자_소수'].apply(convert_from_float_year)
all_synthetic_data['Last_bank_branch_transaction_datetime'] = all_synthetic_data['마지막영업점거래일자_소수'].apply(convert_from_float_year)

all_synthetic_data.drop(columns=['등록일자_소수', '계좌개설일자_소수', '거래일자_소수', '거래재개일자_소수', '마지막ATM거래일자_소수', '마지막영업점거래일자_소수',
                                '거래까지걸린시간_소수', '거래까지걸린시간_ATM_소수', '거래까지걸린시간_영업점_소수', '거래당시연차', '등록부터 계좌개설까지 걸린시간_소수', '등록당시나이'], inplace=True)

In [11]:
all_synthetic_data['Customer_personal_identifier'] = '홍길동'
all_synthetic_data['Customer_identification_number'] = 'aaaaaa-aaaaaaa'
all_synthetic_data['Account_account_number'] = 'aaaaaaaaaa'
all_synthetic_data['IP_Address'] = '38.117.123.196'
all_synthetic_data['MAC_Address'] = '5e:76:37:86:60:c3'
all_synthetic_data['Location'] = '서울특별시 강서구 가양동 37.568238 126.845059'
all_synthetic_data['Recipient_Account_Number'] = 'aaaaaaaaaa'

In [12]:
all_synthetic_data = all_synthetic_data.reindex(columns=train_all.columns)
all_synthetic_data.drop(columns='ID', inplace=True)
all_synthetic_data.to_csv(file_path+'/syn_submission.csv', encoding='UTF-8-sig', index=False)

## 버전 맞추기

## 데이터 불러오기

In [3]:
train = pd.read_csv(file_path + r'\train.csv')
test = pd.read_csv(file_path + r'\test.csv')

In [4]:
train = train.drop(columns="ID")

In [5]:
synthetic_data_ctgan = pd.read_csv(file_path + r"\only_ctgan.csv")
synthetic_data_others = pd.read_csv(file_path + r'\others.csv')

In [6]:
print(train.shape, test.shape, synthetic_data_ctgan.shape, synthetic_data_others.shape)

(120000, 63) (120000, 63) (3600, 63) (6000, 63)


## syn_submission 만들기

> val 분리

In [8]:
X_train, y_train = train.drop(columns="Fraud_Type"), train["Fraud_Type"]

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

In [9]:
train = pd.concat((X_train, y_train), axis=1)
val = pd.concat((X_val, y_val), axis=1)

> 'Fraud_Type'이 'm'만 존재하는 customer을 train에서 제거

In [10]:
only_m_customers = train.groupby('Customer_identification_number')['Fraud_Type'].apply(lambda x: set(x) == {'m'}).reset_index()

# Fraud_Type이 'm'만 있는 Customer_identification_number 필터링
only_m_customers = only_m_customers[only_m_customers['Fraud_Type'] == True]['Customer_identification_number']

# 해당 Customer_identification_number와 일치하는 행들을 train에서 제거
train= train[~train['Customer_identification_number'].isin(only_m_customers)]

In [11]:
print(train.shape, test.shape, synthetic_data_ctgan.shape, synthetic_data_others.shape)

(24468, 63) (120000, 63) (3600, 63) (6000, 63)


In [12]:
# 합성데이터 여부 판단하는 파생변수
DFs = [train, val, test, synthetic_data_ctgan, synthetic_data_others]

for i, df in enumerate(DFs):
    if df is synthetic_data_ctgan or df is synthetic_data_others:
        df['check'] = 0
    else:
        df['check'] = 1
        
    df['check'] = df['check'].astype('int64')
    
    DFs[i] = df

> 원본 데이터와 concat

In [13]:
train_total_ctgan = pd.concat([train, synthetic_data_ctgan])
train_total_others = pd.concat([train, synthetic_data_others])

## Feature Engineering

In [14]:
# 이상치 제거를 위한 통계값을 제공하기 위해 copy 생성

origin_train_ctgan = train_total_ctgan.copy()
origin_train_ctgan['Time_difference'] = pd.to_timedelta(origin_train_ctgan['Time_difference'])
origin_train_ctgan['Time_difference_in_days'] = origin_train_ctgan['Time_difference'].dt.total_seconds() / (24 * 3600)

origin_train_others = train_total_others.copy()
origin_train_others['Time_difference'] = pd.to_timedelta(origin_train_others['Time_difference'])
origin_train_others['Time_difference_in_days'] = origin_train_others['Time_difference'].dt.total_seconds() / (24 * 3600)

In [15]:
# test와 val도 copy 생성

test_ctgan = test.copy()
test_others = test.copy()

val_ctgan = val.copy()
val_others = val.copy()

In [16]:
DFs = [train_total_ctgan, test_ctgan, val_ctgan]

for df in DFs:
    
    origin_train = origin_train_ctgan
    
    # -----------------------------------------------------------------------------------시간 존-----------------------------------------------------------------------------------------
    # datetime 형식으로 변환
    df['Customer_registration_datetime'] = pd.to_datetime(df['Customer_registration_datetime'])
    df['Account_creation_datetime'] = pd.to_datetime(df['Account_creation_datetime'])
    df['Transaction_Datetime'] = pd.to_datetime(df['Transaction_Datetime'])
    df['Last_atm_transaction_datetime'] = pd.to_datetime(df['Last_atm_transaction_datetime'])
    df['Last_bank_branch_transaction_datetime'] = pd.to_datetime(df['Last_bank_branch_transaction_datetime'])
    df['Transaction_resumed_date'] = pd.to_datetime(df['Transaction_resumed_date'])
    df['Time_difference'] = pd.to_timedelta(df['Time_difference'])
    
    # 함수 정의: 날짜를 연도.소수 형식으로 변환
    def convert_to_float_year(dt):
        year = dt.year
        start_of_year = pd.Timestamp(year=year, month=1, day=1)
        next_year = pd.Timestamp(year=year + 1, month=1, day=1)
        year_elapsed = (dt - start_of_year).total_seconds()
        year_duration = (next_year - start_of_year).total_seconds()
        fractional_year = year_elapsed / year_duration
        return year + fractional_year
    
    # [No.6, No.26, No.38, No.55, No.56, No.63] 연도.소수 형식으로 변환
    df['등록일자_소수'] = df['Customer_registration_datetime'].apply(convert_to_float_year)
    df['계좌개설일자_소수'] = df['Account_creation_datetime'].apply(convert_to_float_year)
    df['거래일자_소수'] = df['Transaction_Datetime'].apply(convert_to_float_year)
    df['마지막ATM거래일자_소수'] = df['Last_atm_transaction_datetime'].apply(convert_to_float_year)
    df['마지막영업점거래일자_소수'] = df['Last_bank_branch_transaction_datetime'].apply(convert_to_float_year)
    df['거래재개일자_소수'] = df['Transaction_resumed_date'].apply(convert_to_float_year)
    
    # ------------------------------------------------------------------------------------빼기 존-----------------------------------------------------------------------------------------
    # [No.2]
    df['거래당시나이'] = df['거래일자_소수'] - df['Customer_Birthyear']
    df['거래재개부터 거래까지 걸린시간'] = (df['거래일자_소수'] - df['거래재개일자_소수'])*365
    df['ATM부터 거래까지 걸린시간'] = (df['거래일자_소수'] - df['마지막ATM거래일자_소수'])*365
    df['영업점부터 거래까지 걸린시간'] = (df['거래일자_소수'] - df['마지막영업점거래일자_소수'])*365
    
    # [No.52, No.53] 일 단위로 변환
    df['Time_difference_in_days'] = df['Time_difference'].dt.total_seconds() / (24 * 3600)
    
    # 시간 이상치 처리
    mean_value = origin_train['Time_difference_in_days'].mean()
    df['Time_difference_in_days'] = df['Time_difference_in_days'].apply(
        lambda x: mean_value if x > 365 else x
    )

    min_positive_value = origin_train['Time_difference_in_days'][origin_train['Time_difference_in_days'] >= 0].min()
    df['Time_difference_in_days'] = df['Time_difference_in_days'].apply(
        lambda x: min_positive_value if x < 0 else x
    )
    df["Distance/Time"] = df["Distance"] / df["Time_difference_in_days"]
    
    # ------------------------------------------------------------------------------------더하기 존-----------------------------------------------------------------------------------------
    # [No.8, No.9, No.10, No.11] // [No.16~No.21]  시리즈물들 합산.
    df['Customer_flag_change_of_authentication'] = df['Customer_flag_change_of_authentication_1'] + df['Customer_flag_change_of_authentication_2'] + df['Customer_flag_change_of_authentication_3'] + df['Customer_flag_change_of_authentication_4']
    df['Customer_flag_terminal_malicious_behavior'] = df['Customer_flag_terminal_malicious_behavior_1'] + df['Customer_flag_terminal_malicious_behavior_2'] + df['Customer_flag_terminal_malicious_behavior_3'] + df['Customer_flag_terminal_malicious_behavior_4'] + df['Customer_flag_terminal_malicious_behavior_5'] + df['Customer_flag_terminal_malicious_behavior_6']
    df['7_days'] = df['Customer_inquery_atm_limit'] + df['Customer_increase_atm_limit'] + df['Unused_terminal_status'] + df['Flag_deposit_more_than_tenMillion'] + df['Unused_terminal_status']
 
    # ------------------------------------------------------------------------------------나누기 존-----------------------------------------------------------------------------------------
    df["이체금액/최대지출"] = df.apply(lambda row: row['Transaction_Amount'] / row['Account_one_month_max_amount'] if row['Account_one_month_max_amount'] != 0 else 0, axis=1)
    df['이체금액/초기금액'] = df.apply(lambda row: row['Transaction_Amount'] / row['Account_initial_balance'] if row['Account_initial_balance'] != 0 else 0, axis=1)
    
    df['36/34'] = df.apply(lambda row: row['Account_dawn_one_month_max_amount'] / row['Account_one_month_max_amount'] if row['Account_one_month_max_amount'] != 0 else 0, axis=1)
    df['35/34'] = df.apply(lambda row: row['Account_one_month_std_dev'] / row['Account_one_month_max_amount'] if row['Account_one_month_max_amount'] != 0 else 0, axis=1)
    df['37/36'] = df.apply(lambda row: row['Account_dawn_one_month_std_dev'] / row['Account_dawn_one_month_max_amount'] if row['Account_dawn_one_month_max_amount'] != 0 else 0, axis=1)
    
    # [No.7]순서가 중요한 Label encoding 임의로 수동 부여
    def cus_cre_rat(x):
        if x == 'S':
            return 0
        elif x == 'A':
            return 1
        elif x == 'B':
            return 2
        elif x == 'C':
            return 3
        elif x == 'D':
            return 4
        else:
            return 5
            
    df['Customer_credit_rating'] = df['Customer_credit_rating'].apply(cus_cre_rat)  

    # ----------------------------------------시나리오 기반 인코딩 ----------------------------------------
    # 소액 거래 판단 기준 (최근 1개월 거래 평균 금액이 작고, 현재 이체 금액이 높을 경우)
    # 평소에 소액거래 하는사람 :  중앙값의 절대값(크기)가 작은 사람..?
    # 중앙값이 평균 * 0.1
    df['소액_고액_이체'] = ((df['Account_one_month_max_amount'] * 0.4 > df['Account_one_month_std_dev'] ) & 
                    (abs(df['Transaction_Amount']) > df['Account_one_month_max_amount'] * 1.5) & 
                    (df['Transaction_history_with_the_account'] == 0)).astype(int)
    
    # 평균 거래 금액에서 벗어난 경우
    df['비정상_거래'] = ((abs(df['Transaction_Amount']) > df['Account_one_month_std_dev'] * 2) & 
                 (df['Transaction_history_with_the_account'] == 0)).astype(int)
    
    
    df['소액_고액_이체_dawn'] = ((df['Account_dawn_one_month_max_amount'] * 0.4 > df['Account_dawn_one_month_std_dev'] ) & 
                    (abs(df['Transaction_Amount']) > df['Account_dawn_one_month_max_amount'] * 1.5) & 
                    (df['Transaction_history_with_the_account'] == 0)).astype(int)

    # 평균 거래 금액에서 벗어난 경우
    df['비정상_거래_dawn'] = ((abs(df['Transaction_Amount']) > df['Account_dawn_one_month_std_dev'] * 2) & 
                 (df['Transaction_history_with_the_account'] == 0)).astype(int)
    
    
    df['소액_고액_이체_distance'] = abs(df['Account_one_month_max_amount'] - df['Account_one_month_std_dev'] )
    df['소액_고액_이체_dawn_distance'] = abs(df['Account_dawn_one_month_max_amount'] - df['Account_dawn_one_month_std_dev'] )
    
    ##################################
    # 시나리오 1: 미사용 단말 + 특정 금액 이상 + 기존에 거래한 적 없는 계좌로 송금
    df['Unused_Terminal_High_Amount_New_Account'] = (
        (df['Unused_terminal_status'] == 1) &  # 미사용 단말
        (abs(df['Transaction_Amount']) >= 1000000) &  # 특정 금액 이상 (예: 100만원)
        (df['Transaction_history_with_the_account'] == 0)  # 기존에 거래한 적 없는 계좌
    ).astype(int)

    
    # 시나리오 3: 이상징후 인자를 기반으로 피싱 의심행위 분석
    df['Phishing_Suspicion_Indicator'] = (
        (df['Customer_flag_change_of_authentication_1'] == 1) |  # 금융/공동인증서 재발급 여부
        (df['Customer_flag_change_of_authentication_2'] == 1) |  # 사설인증서 발급 여부
        (df['Customer_flag_change_of_authentication_3'] == 1) |  # 보안카드 및 OTP 발급 여부
        (df['Recipient_account_suspend_status'] == 1) |  # 수취계좌의 거래중지 여부
        (abs(df['Transaction_Amount']) >= 10000000)  # 이체 금액이 1천만 원 이상
    ).astype(int)

    # inf 처리
    for col in [
        'Distance/Time', 
        '이체금액/최대지출'
        ]: 
        # 각 컬럼의 최대값과 최소값 계산 (inf, -inf 제외)
        max_val = train_total_ctgan.loc[train_total_ctgan[col] != np.inf, col].max()
        min_val = train_total_ctgan.loc[train_total_ctgan[col] != -np.inf, col].min()
        
        # inf와 -inf 값을 최대값과 최소값으로 대체
        df[col].replace(np.inf, max_val, inplace=True)
        df[col].replace(-np.inf, min_val, inplace=True)
    
    # 금액 이상치 처리
    def handle_outliers(df, col, origin_df, n_std=3):
        mean = origin_df[col].mean()
        std = origin_df[col].std()
        
        # z-score 계산
        z_scores = (df[col] - mean) / std
        
        # n_std를 넘는 z-score를 가진 값을 n_std에 해당하는 원본 값으로 대체
        series = df[col].mask(z_scores > n_std, mean + n_std * std)
        series = series.mask(z_scores < -n_std, mean - n_std * std)
        
        return series

    cost_cols = [
        'Account_initial_balance', 'Account_balance', 'Account_remaining_amount_daily_limit_exceeded', 'Account_one_month_max_amount', 'Transaction_Amount'
    ]

    for col in cost_cols:
        df[col] = handle_outliers(df, col, origin_train)

    #------------------------------------------------------------------------------------drop 존-----------------------------------------------------------------------------------------
    # 소수로 바꿔 준 6개의 컬럼 drop
    df.drop(columns=['Time_difference', 'Customer_registration_datetime', 'Account_creation_datetime', 'Last_atm_transaction_datetime',
                     'Transaction_Datetime', 'Last_bank_branch_transaction_datetime','Transaction_resumed_date', 'Customer_Birthyear', 
                     '등록일자_소수', '계좌개설일자_소수', '거래일자_소수', '마지막ATM거래일자_소수', '마지막영업점거래일자_소수', '거래재개일자_소수',
                     ], inplace=True)
    print('인코딩 완료했습니다.')

인코딩 완료했습니다.
인코딩 완료했습니다.
인코딩 완료했습니다.


In [17]:
DFs = [train_total_others, test_others, val_others]

for df in DFs:
    
    origin_train = origin_train_others
    
    # -----------------------------------------------------------------------------------시간 존-----------------------------------------------------------------------------------------
    # datetime 형식으로 변환
    df['Customer_registration_datetime'] = pd.to_datetime(df['Customer_registration_datetime'])
    df['Account_creation_datetime'] = pd.to_datetime(df['Account_creation_datetime'])
    df['Transaction_Datetime'] = pd.to_datetime(df['Transaction_Datetime'])
    df['Last_atm_transaction_datetime'] = pd.to_datetime(df['Last_atm_transaction_datetime'])
    df['Last_bank_branch_transaction_datetime'] = pd.to_datetime(df['Last_bank_branch_transaction_datetime'])
    df['Transaction_resumed_date'] = pd.to_datetime(df['Transaction_resumed_date'])
    df['Time_difference'] = pd.to_timedelta(df['Time_difference'])
    
    # 함수 정의: 날짜를 연도.소수 형식으로 변환
    def convert_to_float_year(dt):
        year = dt.year
        start_of_year = pd.Timestamp(year=year, month=1, day=1)
        next_year = pd.Timestamp(year=year + 1, month=1, day=1)
        year_elapsed = (dt - start_of_year).total_seconds()
        year_duration = (next_year - start_of_year).total_seconds()
        fractional_year = year_elapsed / year_duration
        return year + fractional_year
    
    # [No.6, No.26, No.38, No.55, No.56, No.63] 연도.소수 형식으로 변환
    df['등록일자_소수'] = df['Customer_registration_datetime'].apply(convert_to_float_year)
    df['계좌개설일자_소수'] = df['Account_creation_datetime'].apply(convert_to_float_year)
    df['거래일자_소수'] = df['Transaction_Datetime'].apply(convert_to_float_year)
    df['마지막ATM거래일자_소수'] = df['Last_atm_transaction_datetime'].apply(convert_to_float_year)
    df['마지막영업점거래일자_소수'] = df['Last_bank_branch_transaction_datetime'].apply(convert_to_float_year)
    df['거래재개일자_소수'] = df['Transaction_resumed_date'].apply(convert_to_float_year)
    
    # ------------------------------------------------------------------------------------빼기 존-----------------------------------------------------------------------------------------
    # [No.2]
    df['거래당시나이'] = df['거래일자_소수'] - df['Customer_Birthyear']
    df['거래재개부터 거래까지 걸린시간'] = (df['거래일자_소수'] - df['거래재개일자_소수'])*365
    df['ATM부터 거래까지 걸린시간'] = (df['거래일자_소수'] - df['마지막ATM거래일자_소수'])*365
    df['영업점부터 거래까지 걸린시간'] = (df['거래일자_소수'] - df['마지막영업점거래일자_소수'])*365
    
    # [No.52, No.53] 일 단위로 변환
    df['Time_difference_in_days'] = df['Time_difference'].dt.total_seconds() / (24 * 3600)
    
    # 시간 이상치 처리
    mean_value = origin_train['Time_difference_in_days'].mean()
    df['Time_difference_in_days'] = df['Time_difference_in_days'].apply(
        lambda x: mean_value if x > 365 else x
    )

    min_positive_value = origin_train['Time_difference_in_days'][origin_train['Time_difference_in_days'] >= 0].min()
    df['Time_difference_in_days'] = df['Time_difference_in_days'].apply(
        lambda x: min_positive_value if x < 0 else x
    )
    df["Distance/Time"] = df["Distance"] / df["Time_difference_in_days"]
    
    # ------------------------------------------------------------------------------------더하기 존-----------------------------------------------------------------------------------------
    # [No.8, No.9, No.10, No.11] // [No.16~No.21]  시리즈물들 합산.
    df['Customer_flag_change_of_authentication'] = df['Customer_flag_change_of_authentication_1'] + df['Customer_flag_change_of_authentication_2'] + df['Customer_flag_change_of_authentication_3'] + df['Customer_flag_change_of_authentication_4']
    df['Customer_flag_terminal_malicious_behavior'] = df['Customer_flag_terminal_malicious_behavior_1'] + df['Customer_flag_terminal_malicious_behavior_2'] + df['Customer_flag_terminal_malicious_behavior_3'] + df['Customer_flag_terminal_malicious_behavior_4'] + df['Customer_flag_terminal_malicious_behavior_5'] + df['Customer_flag_terminal_malicious_behavior_6']
    df['7_days'] = df['Customer_inquery_atm_limit'] + df['Customer_increase_atm_limit'] + df['Unused_terminal_status'] + df['Flag_deposit_more_than_tenMillion'] + df['Unused_terminal_status']
 
    # ------------------------------------------------------------------------------------나누기 존-----------------------------------------------------------------------------------------
    df["이체금액/최대지출"] = df.apply(lambda row: row['Transaction_Amount'] / row['Account_one_month_max_amount'] if row['Account_one_month_max_amount'] != 0 else 0, axis=1)
    df['이체금액/초기금액'] = df.apply(lambda row: row['Transaction_Amount'] / row['Account_initial_balance'] if row['Account_initial_balance'] != 0 else 0, axis=1)
    
    df['36/34'] = df.apply(lambda row: row['Account_dawn_one_month_max_amount'] / row['Account_one_month_max_amount'] if row['Account_one_month_max_amount'] != 0 else 0, axis=1)
    df['35/34'] = df.apply(lambda row: row['Account_one_month_std_dev'] / row['Account_one_month_max_amount'] if row['Account_one_month_max_amount'] != 0 else 0, axis=1)
    df['37/36'] = df.apply(lambda row: row['Account_dawn_one_month_std_dev'] / row['Account_dawn_one_month_max_amount'] if row['Account_dawn_one_month_max_amount'] != 0 else 0, axis=1)
    
    # [No.7]순서가 중요한 Label encoding 임의로 수동 부여
    def cus_cre_rat(x):
        if x == 'S':
            return 0
        elif x == 'A':
            return 1
        elif x == 'B':
            return 2
        elif x == 'C':
            return 3
        elif x == 'D':
            return 4
        else:
            return 5
            
    df['Customer_credit_rating'] = df['Customer_credit_rating'].apply(cus_cre_rat)  

    # ----------------------------------------시나리오 기반 인코딩 ----------------------------------------
    # 소액 거래 판단 기준 (최근 1개월 거래 평균 금액이 작고, 현재 이체 금액이 높을 경우)
    # 평소에 소액거래 하는사람 :  중앙값의 절대값(크기)가 작은 사람..?
    # 중앙값이 평균 * 0.1
    df['소액_고액_이체'] = ((df['Account_one_month_max_amount'] * 0.4 > df['Account_one_month_std_dev'] ) & 
                    (abs(df['Transaction_Amount']) > df['Account_one_month_max_amount'] * 1.5) & 
                    (df['Transaction_history_with_the_account'] == 0)).astype(int)
    
    # 평균 거래 금액에서 벗어난 경우
    df['비정상_거래'] = ((abs(df['Transaction_Amount']) > df['Account_one_month_std_dev'] * 2) & 
                 (df['Transaction_history_with_the_account'] == 0)).astype(int)
    
    
    df['소액_고액_이체_dawn'] = ((df['Account_dawn_one_month_max_amount'] * 0.4 > df['Account_dawn_one_month_std_dev'] ) & 
                    (abs(df['Transaction_Amount']) > df['Account_dawn_one_month_max_amount'] * 1.5) & 
                    (df['Transaction_history_with_the_account'] == 0)).astype(int)

    # 평균 거래 금액에서 벗어난 경우
    df['비정상_거래_dawn'] = ((abs(df['Transaction_Amount']) > df['Account_dawn_one_month_std_dev'] * 2) & 
                 (df['Transaction_history_with_the_account'] == 0)).astype(int)
    
    
    df['소액_고액_이체_distance'] = abs(df['Account_one_month_max_amount'] - df['Account_one_month_std_dev'] )
    df['소액_고액_이체_dawn_distance'] = abs(df['Account_dawn_one_month_max_amount'] - df['Account_dawn_one_month_std_dev'] )
    
    ##################################
    # 시나리오 1: 미사용 단말 + 특정 금액 이상 + 기존에 거래한 적 없는 계좌로 송금
    df['Unused_Terminal_High_Amount_New_Account'] = (
        (df['Unused_terminal_status'] == 1) &  # 미사용 단말
        (abs(df['Transaction_Amount']) >= 1000000) &  # 특정 금액 이상 (예: 100만원)
        (df['Transaction_history_with_the_account'] == 0)  # 기존에 거래한 적 없는 계좌
    ).astype(int)

    
    # 시나리오 3: 이상징후 인자를 기반으로 피싱 의심행위 분석
    df['Phishing_Suspicion_Indicator'] = (
        (df['Customer_flag_change_of_authentication_1'] == 1) |  # 금융/공동인증서 재발급 여부
        (df['Customer_flag_change_of_authentication_2'] == 1) |  # 사설인증서 발급 여부
        (df['Customer_flag_change_of_authentication_3'] == 1) |  # 보안카드 및 OTP 발급 여부
        (df['Recipient_account_suspend_status'] == 1) |  # 수취계좌의 거래중지 여부
        (abs(df['Transaction_Amount']) >= 10000000)  # 이체 금액이 1천만 원 이상
    ).astype(int)

    # inf 처리
    for col in [
        'Distance/Time', 
        '이체금액/최대지출'
        ]: 
        # 각 컬럼의 최대값과 최소값 계산 (inf, -inf 제외)
        max_val = train_total_others.loc[train_total_others[col] != np.inf, col].max()
        min_val = train_total_others.loc[train_total_others[col] != -np.inf, col].min()
        
        # inf와 -inf 값을 최대값과 최소값으로 대체
        df[col].replace(np.inf, max_val, inplace=True)
        df[col].replace(-np.inf, min_val, inplace=True)
    
    # 금액 이상치 처리
    def handle_outliers(df, col, origin_df, n_std=3):
        mean = origin_df[col].mean()
        std = origin_df[col].std()
        
        # z-score 계산
        z_scores = (df[col] - mean) / std
        
        # n_std를 넘는 z-score를 가진 값을 n_std에 해당하는 원본 값으로 대체
        series = df[col].mask(z_scores > n_std, mean + n_std * std)
        series = series.mask(z_scores < -n_std, mean - n_std * std)
        
        return series

    cost_cols = [
        'Account_initial_balance', 'Account_balance', 'Account_remaining_amount_daily_limit_exceeded', 'Account_one_month_max_amount', 'Transaction_Amount'
    ]

    for col in cost_cols:
        df[col] = handle_outliers(df, col, origin_train)

    #------------------------------------------------------------------------------------drop 존-----------------------------------------------------------------------------------------
    # 소수로 바꿔 준 6개의 컬럼 drop
    df.drop(columns=['Time_difference', 'Customer_registration_datetime', 'Account_creation_datetime', 'Last_atm_transaction_datetime',
                     'Transaction_Datetime', 'Last_bank_branch_transaction_datetime','Transaction_resumed_date', 'Customer_Birthyear', 
                     '등록일자_소수', '계좌개설일자_소수', '거래일자_소수', '마지막ATM거래일자_소수', '마지막영업점거래일자_소수', '거래재개일자_소수',
                     ], inplace=True)
    print('인코딩 완료했습니다.')

인코딩 완료했습니다.
인코딩 완료했습니다.
인코딩 완료했습니다.


In [18]:
print(train_total_ctgan.shape, val_ctgan.shape, test_ctgan.shape, train_total_others.shape, val_others.shape, test_others.shape)

(28068, 78) (12000, 78) (120000, 78) (30468, 78) (12000, 78) (120000, 78)


In [19]:
DFs = [
    [train_total_ctgan, val_ctgan, test_ctgan],
    [train_total_others, val_others, test_others]
]

In [20]:
# aggregation 기반 파생변수
def anomal(x):
    if x != 'm':
        return 1
    else:
        return 0

for i, (train, val, test) in enumerate(DFs):
    
    train['anomal'] = train['Fraud_Type'].apply(anomal)

    cols = [
            'Account_account_type', 
            'Channel',
            'Access_Medium'
            ]

    for col in cols:
        df_cnt = train.groupby([col])['anomal'].count().reset_index()
        df_sum = train.groupby([col])['anomal'].sum().reset_index()
        df_ratio = pd.merge(df_cnt, df_sum, on=col)
        df_ratio[f"{col}_ratio"] = df_ratio["anomal_y"]/df_ratio['anomal_x']
        if f"{col}_ratio" not in train.columns:
            train = pd.merge(train, df_ratio[[col, f'{col}_ratio']], on=col, how='left')
            test = pd.merge(test, df_ratio[[col, f'{col}_ratio']], on=col, how='left')
            val = pd.merge(val, df_ratio[[col, f'{col}_ratio']], on=col, how='left')
            

    train.drop(columns='anomal', inplace=True)
    DFs[i] = [train, val, test]

In [21]:
train_ctgan_x = DFs[0][0].drop(columns=['Fraud_Type'])
train_ctgan_y = DFs[0][0]['Fraud_Type']
val_ctgan_x = DFs[0][1].drop(columns=['Fraud_Type'])
val_ctgan_y = DFs[0][1]['Fraud_Type']
test_ctgan_x = DFs[0][2].drop(columns=['ID'])

train_others_x = DFs[1][0].drop(columns=['Fraud_Type'])
train_others_y = DFs[1][0]['Fraud_Type']
val_others_x = DFs[1][1].drop(columns=['Fraud_Type'])
val_others_y = DFs[1][1]['Fraud_Type']
test_others_x = DFs[1][2].drop(columns=['ID'])

In [22]:
print(train_ctgan_x.shape, val_ctgan_x.shape, test_ctgan_x.shape, train_others_x.shape, val_others_x.shape, test_others_x.shape)

(28068, 80) (12000, 80) (120000, 80) (30468, 80) (12000, 80) (120000, 80)


## 컬럼 드랍

In [23]:
drop_columns = [
                'Customer_identification_number', "Customer_personal_identifier", "Account_account_number", 'Recipient_Account_Number', 
                "First_time_iOS_by_vulnerable_user", 
                "Customer_Gender",
                "Another_Person_Account",
                'Error_Code',
                'Operating_System', 
                'Account_one_month_std_dev',
                'Account_dawn_one_month_std_dev',
                'Account_amount_daily_limit',
                "IP_Address", "MAC_Address", "Location", 
                ]

for column in drop_columns:
    train_ctgan_x = train_ctgan_x.drop(column, axis=1)
    val_ctgan_x = val_ctgan_x.drop(column, axis=1)
    test_ctgan_x = test_ctgan_x.drop(column, axis=1)
    train_others_x = train_others_x.drop(column, axis=1)
    val_others_x = val_others_x.drop(column, axis=1)
    test_others_x = test_others_x.drop(column, axis=1)

In [24]:
x_DFs = [
    [train_ctgan_x, val_ctgan_x, test_ctgan_x],
    [train_others_x, val_others_x, test_others_x]
]

## 컬럼 인코딩

In [25]:
 # 원핫 인코딩을 할 컬럼 리스트
one_hot_columns = [
                   'Channel', 
                   'Access_Medium',
                   'Account_account_type',
                   'Customer_loan_type'
                   ]

for i, (train_x, val_x, test_x) in enumerate(x_DFs):
    
    # 모든 데이터를 합친 후 원핫 인코딩
    tem = pd.concat((pd.concat((train_x, val_x)), test_x))
    tem = pd.get_dummies(tem, columns=one_hot_columns)

    # 원래의 train, val, test 데이터셋으로 분리
    train_x, val_x, test_x = tem[:len(train_x)], tem[len(train_x):len(train_x)+len(val_x)], tem[len(train_x)+len(val_x):]

    train_x_encoded = train_x.copy()
    val_x_encoded = val_x.copy()
    test_x_encoded = test_x.copy()

    object_columns = train_x.select_dtypes(include=['object']).columns
    ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    train_x_encoded[object_columns] = ordinal_encoder.fit_transform(train_x[object_columns])

    feature_order = train_x_encoded.columns.tolist()

    test_x_encoded[object_columns] = ordinal_encoder.transform(test_x[object_columns])
    val_x_encoded[object_columns] = ordinal_encoder.transform(val_x[object_columns])

    # 특성 순서 맞추기 및 데이터 타입 일치
    test_x_encoded = test_x_encoded[feature_order]
    for col in feature_order:
        test_x_encoded[col] = test_x_encoded[col].astype(train_x_encoded[col].dtype)

    val_x_encoded = val_x_encoded[feature_order]
    for col in feature_order:
        val_x_encoded[col] = val_x_encoded[col].astype(train_x_encoded[col].dtype)
        
    x_DFs[i] = [train_x_encoded, val_x_encoded, test_x_encoded]

## target 인코딩

In [26]:
le_subclass = LabelEncoder()
train_ctgan_y_encoded = le_subclass.fit_transform(train_ctgan_y)
val_ctgan_y_encoded = le_subclass.fit_transform(val_ctgan_y)
train_others_y_encoded = le_subclass.fit_transform(train_others_y)
val_others_y_encoded = le_subclass.fit_transform(val_others_y)

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

원래 레이블: a, 변환된 숫자: 0
원래 레이블: b, 변환된 숫자: 1
원래 레이블: c, 변환된 숫자: 2
원래 레이블: d, 변환된 숫자: 3
원래 레이블: e, 변환된 숫자: 4
원래 레이블: f, 변환된 숫자: 5
원래 레이블: g, 변환된 숫자: 6
원래 레이블: h, 변환된 숫자: 7
원래 레이블: i, 변환된 숫자: 8
원래 레이블: j, 변환된 숫자: 9
원래 레이블: k, 변환된 숫자: 10
원래 레이블: l, 변환된 숫자: 11
원래 레이블: m, 변환된 숫자: 12


## XGBoost OOF + LGBM OOF + random states

In [27]:
# 공통 파라미터

FOLDS = 10
random_states = [0, 22, 42, 1215]  # 사용할 random_state 값들

train_ctgan_x_encoded = x_DFs[0][0]
val_ctgan_x_encoded = x_DFs[0][1]
test_ctgan_x_encoded = x_DFs[0][2]

train_others_x_encoded = x_DFs[1][0]
val_others_x_encoded = x_DFs[1][1]
test_others_x_encoded = x_DFs[1][2]

In [28]:
pd.options.display.max_rows = 61
pd.options.display.max_columns = 20

In [29]:
print(train_ctgan_x_encoded.shape, val_ctgan_x_encoded.shape, test_ctgan_x_encoded.shape, train_others_x_encoded.shape, val_others_x_encoded.shape, test_others_x_encoded.shape)

(28068, 81) (12000, 81) (120000, 81) (30468, 81) (12000, 81) (120000, 81)


In [30]:
# 압축 파일 경로
model_save_path = file_path + "/saved_models.zip"

# 압축 해제할 디렉토리 설정
extract_dir = file_path + "/extracted_models"

with zipfile.ZipFile(model_save_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"압축 해제 완료. 파일이 {extract_dir}에 저장되었습니다.")

압축 해제 완료. 파일이 C:\Users\james\Desktop\2024-8./extracted_models에 저장되었습니다.


In [34]:
# 기본 설정
X_train = train_ctgan_x_encoded
y_train = train_ctgan_y_encoded
X_val = val_ctgan_x_encoded
y_val = val_ctgan_y_encoded
X_test = test_ctgan_x_encoded

# 결과 저장용 리스트
all_macro_f1_scores = []
all_class_f1_scores = []

# 테스트 데이터 예측 결과 저장용 배열 초기화
final_xgb_ctgan_test_preds_proba = np.zeros((X_test.shape[0], len(np.unique(y_train))))

# 다양한 random_state로 반복
for r_state in random_states:
    print(f"## Random State: {r_state} ##\n")
    
    # Stratified K-Fold 설정
    folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=r_state)

    params = {
    'objective': 'multi:softprob',  # 'multiclass' -> 'multi:softprob' for XGBoost
    'num_class': len(np.unique(y_train)),  # Same as in LightGBM
    'learning_rate': 0.044617907759625222,  # Same as in LightGBM
    'random_state': 42,  # XGBoost uses 'random_state' as well
    'eval_metric': 'mlogloss',  # 'metric': 'multi_logloss' -> 'eval_metric': 'mlogloss'
    'early_stopping_rounds': 100,  # Early stopping rounds are passed to `XGB.train` or `fit`
    'n_jobs': -1,  # Same as in LightGBM
    'alpha': 2.6902660114996324e-07,
    'lambda': 2.0514464638605287e-08,
    # 'gamma': 6.888304982078908e-08,
    'n_estimators': 2500,
    'min_child_weight': 0.0036996214238581523,
    "tree_method": "gpu_hist",  # GPU 사용
    "gpu_id": 0,
}

    # F1 스코어 및 중요도 저장용 리스트 초기화
    macro_f1_score_list = []
    class_f1_scores = []

    # 각 클래스의 샘플 수 계산
    class_counts = np.bincount(y_train)
    total_samples = len(y_train)
    class_weights = total_samples / (len(class_counts) * class_counts)

    # 가중치 배열 생성
    weights = np.array([class_weights[label] for label in y_train])

    for idx, (train_idx, _) in enumerate(folds.split(X_train, y_train)):
        print('#' * 40, f'폴드 {idx+1} / {folds.n_splits}', "#" * 40)
        X_tr, y_tr = X_train.iloc[train_idx], y_train[train_idx]
        weights_tr = weights[train_idx]
    
        xgb_model = xgb.XGBClassifier(**params, use_label_encoder=False)
        
        xgb_model.load_model(extract_dir+'/saved_models'+f'/xgb_model_ctgan_{r_state}_fold_{idx+1}.json')
        
        # 테스트 데이터 예측 (여러 random_state 결과를 평균)
        final_xgb_ctgan_test_preds_proba += xgb_model.predict_proba(X_test) / (len(random_states)*folds.n_splits)
        
        # 검증 데이터 예측
        val_preds_proba = xgb_model.predict_proba(X_val)
        val_preds = np.argmax(val_preds_proba, axis=1)
        
        # 다중 클래스 F1 스코어 계산 (클래스별 F1 스코어 포함)
        macro_f1 = f1_score(y_val, val_preds, average='macro')
        class_f1 = f1_score(y_val, val_preds, average=None)
        macro_f1_score_list.append(macro_f1)
        class_f1_scores.append(class_f1)
        
        print(f'폴드 {idx+1} Macro F1 score: {macro_f1}\n')
        print(f'폴드 {idx+1} 클래스별 F1 score:\n{classification_report(y_val, val_preds)}\n')

    # 각 random_state에 대한 결과 저장
    avg_macro_f1 = np.mean(macro_f1_score_list)
    all_macro_f1_scores.append(avg_macro_f1)
    all_class_f1_scores.append(class_f1_scores)

    print(f'Random State {r_state} 검증 평균 Macro F1 score: {avg_macro_f1}\n')

# 전체 결과 요약
for i, r_state in enumerate(random_states):
    print(f"## Random State {r_state} 결과 ##")
    print(f"검증 평균 Macro F1 score: {all_macro_f1_scores[i]}")
    print(f"클래스별 F1 score:\n{np.mean(all_class_f1_scores[i], axis=0)}\n")
    print("#" * 80)

## Random State: 0 ##

######################################## 폴드 1 / 10 ########################################
폴드 1 Macro F1 score: 0.6876640344617554

폴드 1 클래스별 F1 score:
              precision    recall  f1-score   support

           0       0.82      0.90      0.86        10
           1       0.69      0.90      0.78        10
           2       0.83      0.50      0.62        10
           3       0.62      0.50      0.56        10
           4       0.75      0.90      0.82        10
           5       0.73      0.80      0.76        10
           6       0.50      0.40      0.44        10
           7       0.62      0.50      0.56        10
           8       0.42      0.50      0.45        10
           9       0.58      0.70      0.64        10
          10       0.75      0.90      0.82        10
          11       0.67      0.60      0.63        10
          12       1.00      1.00      1.00     11880

    accuracy                           1.00     12000
   macro avg

KeyboardInterrupt: 

In [40]:
# 기본 설정
X_train = train_ctgan_x_encoded
y_train = train_ctgan_y_encoded
X_val = val_ctgan_x_encoded
y_val = val_ctgan_y_encoded
X_test = test_ctgan_x_encoded

# 결과 저장용 리스트
all_macro_f1_scores = []

# 테스트 데이터 예측 결과 저장용 배열 초기화
final_lgb_ctgan_test_preds_proba = np.zeros((X_test.shape[0], len(np.unique(y_train))))

# 다양한 random_state로 반복
for r_state in random_states:
    print(f"## Random State: {r_state} ##\n")
    
    # Stratified K-Fold 설정
    folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=r_state)

    # LightGBM 모델 파라미터 설정
    params = {
        'objective': 'multiclass',
        'num_class': len(np.unique(y_train)),
        'learning_rate': 0.01786236294491012,
        'random_state': 42,
        'metric': 'multi_logloss',
        'early_stopping_rounds': 100,
        'n_jobs': -1,
        'verbose': -1,
        'num_leaves': 31,
        'min_data_in_leaf': 27,
        'lambda_l1': 0.00046641762342032746,
        'lambda_l2': 8.228074508440626e-06,
        'min_split_gain': 0.0008507877755254931,
        'min_child_weight': 0.00044655605235435
    }

    # F1 스코어 및 중요도 저장용 리스트 초기화
    macro_f1_score_list = []
    class_f1_scores = []

    # 각 클래스의 샘플 수 계산
    class_counts = np.bincount(y_train)
    total_samples = len(y_train)
    class_weights = total_samples / (len(class_counts) * class_counts)

    # 가중치 배열 생성
    weights = np.array([class_weights[label] for label in y_train])

    for idx, (train_idx, _) in enumerate(folds.split(X_train, y_train)):
        print('#' * 40, f'폴드 {idx+1} / {folds.n_splits}', "#" * 40)
        X_tr, y_tr = X_train.iloc[train_idx], y_train[train_idx]
        weights_tr = weights[train_idx]
        
        d_train = lgb.Dataset(X_tr, label=y_tr, weight=weights_tr)
        d_valid = lgb.Dataset(X_val, label=y_val, reference=d_train)

        lgb_model = lgb.Booster(model_file=extract_dir+'/saved_models'+f'/lgb_model_ctgan_{r_state}_fold_{idx+1}.txt')
        
        # 테스트 데이터 예측 (여러 random_state 결과를 평균)
        final_lgb_ctgan_test_preds_proba += lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration) / (len(random_states)*folds.n_splits)
        
        # 검증 데이터 예측
        val_preds_proba = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration)
        val_preds = np.argmax(val_preds_proba, axis=1)
        
        # 다중 클래스 F1 스코어 계산 (클래스별 F1 스코어 포함)
        macro_f1 = f1_score(y_val, val_preds, average='macro')
        class_f1 = f1_score(y_val, val_preds, average=None)
        macro_f1_score_list.append(macro_f1)
        class_f1_scores.append(class_f1)
        
        print(f'폴드 {idx+1} Macro F1 score: {macro_f1}\n')
        print(f'폴드 {idx+1} 클래스별 F1 score:\n{classification_report(y_val, val_preds)}\n')

    # 각 random_state에 대한 결과 저장
    avg_macro_f1 = np.mean(macro_f1_score_list)
    all_macro_f1_scores.append(avg_macro_f1)
    all_class_f1_scores.append(class_f1_scores)

    print(f'Random State {r_state} 검증 평균 Macro F1 score: {avg_macro_f1}\n')

# 전체 결과 요약
for i, r_state in enumerate(random_states):
    print(f"## Random State {r_state} 결과 ##")
    print(f"검증 평균 Macro F1 score: {all_macro_f1_scores[i]}")
    print(f"클래스별 F1 score:\n{np.mean(all_class_f1_scores[i], axis=0)}\n")
    print("#" * 80)

## Random State: 0 ##

######################################## 폴드 1 / 10 ########################################
폴드 1 Macro F1 score: 0.7044435360820943

폴드 1 클래스별 F1 score:
              precision    recall  f1-score   support

           0       0.62      1.00      0.77        10
           1       0.78      0.70      0.74        10
           2       0.88      0.70      0.78        10
           3       0.71      0.50      0.59        10
           4       0.89      0.80      0.84        10
           5       0.69      0.90      0.78        10
           6       0.67      0.40      0.50        10
           7       0.67      0.60      0.63        10
           8       0.50      0.60      0.55        10
           9       0.55      0.60      0.57        10
          10       0.69      0.90      0.78        10
          11       0.67      0.60      0.63        10
          12       1.00      1.00      1.00     11880

    accuracy                           1.00     12000
   macro avg

In [34]:
# 기본 설정
X_train = train_others_x_encoded
y_train = train_others_y_encoded
X_val = val_others_x_encoded
y_val = val_others_y_encoded
X_test = test_others_x_encoded

# 결과 저장용 리스트
all_macro_f1_scores = []
all_class_f1_scores = []

# 테스트 데이터 예측 결과 저장용 배열 초기화
final_xgb_others_test_preds_proba = np.zeros((X_test.shape[0], len(np.unique(y_train))))

# 다양한 random_state로 반복
for r_state in random_states:
    print(f"## Random State: {r_state} ##\n")
    
    # Stratified K-Fold 설정
    folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=r_state)

    params = {
    'objective': 'multi:softprob',  # 'multiclass' -> 'multi:softprob' for XGBoost
    'num_class': len(np.unique(y_train)),  # Same as in LightGBM
    'learning_rate': 0.05344249441648658,  # Same as in LightGBM
    'max_depth': 6,
    'min_child_weight': 0.016537305039295312,
    'lambda': 4.8134969202861536e-06,
    'alpha': 1.3035009627597196e-08,
    'random_state': 42,  # XGBoost uses 'random_state' as well
    'eval_metric': 'mlogloss',  # 'metric': 'multi_logloss' -> 'eval_metric': 'mlogloss'
    'early_stopping_rounds': 50,  # Early stopping rounds are passed to `XGB.train` or `fit`
    'n_jobs': -1,  # Same as in LightGBM
    'n_estimators': 2500,
    "tree_method": "gpu_hist",  # GPU 사용
    "gpu_id": 0,
}


    # F1 스코어 및 중요도 저장용 리스트 초기화
    macro_f1_score_list = []
    class_f1_scores = []

    # 각 클래스의 샘플 수 계산
    class_counts = np.bincount(y_train)
    total_samples = len(y_train)
    class_weights = total_samples / (len(class_counts) * class_counts)

    # 가중치 배열 생성
    weights = np.array([class_weights[label] for label in y_train])

    for idx, (train_idx, _) in enumerate(folds.split(X_train, y_train)):
        print('#' * 40, f'폴드 {idx+1} / {folds.n_splits}', "#" * 40)
        X_tr, y_tr = X_train.iloc[train_idx], y_train[train_idx]
        weights_tr = weights[train_idx]
    
        xgb_model = xgb.XGBClassifier(**params, use_label_encoder=False)
        
        xgb_model.load_model(extract_dir+'/saved_models'+f'/xgb_model_others_{r_state}_fold_{idx+1}.json')
        
        # 테스트 데이터 예측 (여러 random_state 결과를 평균)
        final_xgb_others_test_preds_proba += xgb_model.predict_proba(X_test) / (len(random_states)*folds.n_splits)
        
        # 검증 데이터 예측
        val_preds_proba = xgb_model.predict_proba(X_val)
        val_preds = np.argmax(val_preds_proba, axis=1)
        
        # 다중 클래스 F1 스코어 계산 (클래스별 F1 스코어 포함)
        macro_f1 = f1_score(y_val, val_preds, average='macro')
        class_f1 = f1_score(y_val, val_preds, average=None)
        macro_f1_score_list.append(macro_f1)
        class_f1_scores.append(class_f1)
        
        print(f'폴드 {idx+1} Macro F1 score: {macro_f1}\n')
        print(f'폴드 {idx+1} 클래스별 F1 score:\n{classification_report(y_val, val_preds)}\n')

    # 각 random_state에 대한 결과 저장
    avg_macro_f1 = np.mean(macro_f1_score_list)
    all_macro_f1_scores.append(avg_macro_f1)
    all_class_f1_scores.append(class_f1_scores)

    print(f'Random State {r_state} 검증 평균 Macro F1 score: {avg_macro_f1}\n')

# 전체 결과 요약
for i, r_state in enumerate(random_states):
    print(f"## Random State {r_state} 결과 ##")
    print(f"검증 평균 Macro F1 score: {all_macro_f1_scores[i]}")
    print(f"클래스별 F1 score:\n{np.mean(all_class_f1_scores[i], axis=0)}\n")
    print("#" * 80)

## Random State: 0 ##

######################################## 폴드 1 / 10 ########################################
[0]	validation_0-mlogloss:2.28301	validation_1-mlogloss:2.27836
[1]	validation_0-mlogloss:2.07616	validation_1-mlogloss:2.06925
[2]	validation_0-mlogloss:1.91269	validation_1-mlogloss:1.90445
[3]	validation_0-mlogloss:1.77630	validation_1-mlogloss:1.76691
[4]	validation_0-mlogloss:1.65918	validation_1-mlogloss:1.64914
[5]	validation_0-mlogloss:1.55700	validation_1-mlogloss:1.54583
[6]	validation_0-mlogloss:1.46601	validation_1-mlogloss:1.45433
[7]	validation_0-mlogloss:1.38482	validation_1-mlogloss:1.37281
[8]	validation_0-mlogloss:1.31020	validation_1-mlogloss:1.29788
[9]	validation_0-mlogloss:1.24204	validation_1-mlogloss:1.22957
[10]	validation_0-mlogloss:1.17928	validation_1-mlogloss:1.16715
[11]	validation_0-mlogloss:1.12173	validation_1-mlogloss:1.10967
[12]	validation_0-mlogloss:1.06815	validation_1-mlogloss:1.05593
[13]	validation_0-mlogloss:1.01818	validation_1-ml

In [35]:
# 기본 설정
X_train = train_others_x_encoded
y_train = train_others_y_encoded
X_val = val_others_x_encoded
y_val = val_others_y_encoded
X_test = test_others_x_encoded

# 결과 저장용 리스트
all_macro_f1_scores = []
all_class_f1_scores = []

# 테스트 데이터 예측 결과 저장용 배열 초기화
final_lgb_others_test_preds_proba = np.zeros((X_test.shape[0], len(np.unique(y_train))))

# 다양한 random_state로 반복
for r_state in random_states:
    print(f"## Random State: {r_state} ##\n")
    
    # Stratified K-Fold 설정
    folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=r_state)

    # LightGBM 모델 파라미터 설정
    params = {
    'objective': 'multiclass',
    'num_class': len(np.unique(y_train)),
    'learning_rate': 0.011734924728949315,
    'random_state': 42,
    'metric': 'multi_logloss',
    'early_stopping_rounds': 100,
    'n_jobs': -1,
    'verbose': -1,
    'num_leaves': 29,
    'min_data_in_leaf': 20, 
    'min_split_gain': 1.4774068878825667e-06,
    'min_child_weight': 4.5237470900890216e-05,
    'lambda_l1': 9.167526090129327e-07,
    'lambda_l2': 2.3892407917397933e-08
    }

    # F1 스코어 및 중요도 저장용 리스트 초기화
    macro_f1_score_list = []
    class_f1_scores = []

    # 각 클래스의 샘플 수 계산
    class_counts = np.bincount(y_train)
    total_samples = len(y_train)
    class_weights = total_samples / (len(class_counts) * class_counts)

    # 가중치 배열 생성
    weights = np.array([class_weights[label] for label in y_train])

    for idx, (train_idx, _) in enumerate(folds.split(X_train, y_train)):
        print('#' * 40, f'폴드 {idx+1} / {folds.n_splits}', "#" * 40)
        X_tr, y_tr = X_train.iloc[train_idx], y_train[train_idx]
        weights_tr = weights[train_idx]
        
        d_train = lgb.Dataset(X_tr, label=y_tr, weight=weights_tr)
        d_valid = lgb.Dataset(X_val, label=y_val, reference=d_train)
        
        lgb_model = lgb.Booster(model_file=extract_dir+'/saved_models'+f'/lgb_model_others_{r_state}_fold_{idx+1}.txt')
        
        # 테스트 데이터 예측 (여러 random_state 결과를 평균)
        final_lgb_others_test_preds_proba += lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration) / (len(random_states)*folds.n_splits)
        
        # 검증 데이터 예측
        val_preds_proba = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration)
        val_preds = np.argmax(val_preds_proba, axis=1)
        
        # 다중 클래스 F1 스코어 계산 (클래스별 F1 스코어 포함)
        macro_f1 = f1_score(y_val, val_preds, average='macro')
        class_f1 = f1_score(y_val, val_preds, average=None)
        macro_f1_score_list.append(macro_f1)
        class_f1_scores.append(class_f1)
        
        print(f'폴드 {idx+1} Macro F1 score: {macro_f1}\n')
        print(f'폴드 {idx+1} 클래스별 F1 score:\n{classification_report(y_val, val_preds)}\n')

    # 각 random_state에 대한 결과 저장
    avg_macro_f1 = np.mean(macro_f1_score_list)
    all_macro_f1_scores.append(avg_macro_f1)
    all_class_f1_scores.append(class_f1_scores)

    print(f'Random State {r_state} 검증 평균 Macro F1 score: {avg_macro_f1}\n')

# 전체 결과 요약
for i, r_state in enumerate(random_states):
    print(f"## Random State {r_state} 결과 ##")
    print(f"검증 평균 Macro F1 score: {all_macro_f1_scores[i]}")
    print(f"클래스별 F1 score:\n{np.mean(all_class_f1_scores[i], axis=0)}\n")
    print("#" * 80)

## Random State: 0 ##

######################################## 폴드 1 / 10 ########################################
폴드 1 Macro F1 score: 0.7455061969814787

폴드 1 클래스별 F1 score:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        10
           1       0.89      0.80      0.84        10
           2       0.75      0.60      0.67        10
           3       0.80      0.80      0.80        10
           4       0.73      0.80      0.76        10
           5       0.69      0.90      0.78        10
           6       0.38      0.30      0.33        10
           7       0.75      0.60      0.67        10
           8       0.80      0.80      0.80        10
           9       0.62      0.50      0.56        10
          10       0.90      0.90      0.90        10
          11       0.67      0.60      0.63        10
          12       1.00      1.00      1.00     11880

    accuracy                           1.00     12000
   macro avg

## 최종 예측 및 제출

In [36]:
final_test_preds = 0.35*final_lgb_others_test_preds_proba + 0.15*final_xgb_others_test_preds_proba+0.2*final_lgb_ctgan_test_preds_proba+0.3*final_xgb_ctgan_test_preds_proba
final_test_preds = np.argmax(final_test_preds, axis=1)

clf_submission = pd.read_csv(file_path + r"\sample_submission.csv")
clf_submission["Fraud_Type"] = le_subclass.inverse_transform(final_test_preds)
clf_submission["Fraud_Type"].value_counts()

Fraud_Type
m    67286
f     5348
e     5186
h     5128
a     4910
k     4757
l     4702
j     4650
b     4585
i     4060
g     3430
c     3043
d     2915
Name: count, dtype: int64

In [37]:
clf_submission.to_csv(file_path + './clf_submission_final.csv', encoding='UTF-8-sig', index=False)